In [1]:
import pandas as pd
import numpy as np
from ipyslickgrid import show_grid
from ipywidgets import VBox, IntSlider
from plotly.graph_objs import Bar
from plotly.graph_objs.layout import Annotation

from simple.chart import chartParallel, chartFigure, updateFigure
from simple.backtest import npBacktestMarket, getProfit
from simple.pretty import pp

pd.options.display.float_format = '{:,.1f}'.format
np.set_printoptions(suppress=True, linewidth=140, edgeitems=5, precision=4)

In [2]:
# Parallel coordinates chart
R = pd.read_parquet('result.parquet')
r = R.drop(columns=['Pred', 'Target', 'PnL'])
chartParallel(r, inverse=['RMSE(t)', 'RMSE(v)', 'Horizon'])

FigureWidget({
    'data': [{'dimensions': [{'label': 'Horizon',
                              'range': [24, 1…

In [3]:
# Profit table for all horizons and threshold
b = 12    # skip lower thresholds due high fees
E = R.PnL.apply(pd.Series).T
P = pd.concat([E.iloc[b::4], pd.DataFrame(R.PRatio).T, pd.DataFrame(R.PMean).T, pd.DataFrame(R.PMax).T])
display(pp(P, h_subset=(P.iloc[-4:].index, slice(None))))

Horizon,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
12,-837.8,"-1,360.7","-2,476.9","-2,614.4","-2,302.0","-1,794.4","-1,030.6",-84.7,-86.3,-132.8,-83.9,-69.5,-77.2,-66.3,-77.8,-61.7,-47.7,-47.4,-52.3,-61.4,-80.4,-35.9,-26.3,-20.4
16,-157.5,-48.8,-306.6,-146.9,-42.9,-74.6,-10.0,-8.3,-1.1,-47.0,-30.9,-30.6,-20.9,-25.1,-39.9,-32.8,-0.8,-7.1,-26.4,-34.6,-23.6,-13.5,7.5,7.6
20,-77.1,-27.6,-56.3,-66.4,-9.8,-40.9,1.9,-5.4,2.4,-23.7,-12.8,-16.2,15.4,-3.7,-32.5,-17.7,9.2,-1.1,-2.8,-25.3,-12.9,-2.9,7.0,1.1
24,-63.3,-26.9,-28.6,-44.5,-0.4,-27.0,9.8,8.9,-0.6,-8.6,-11.5,-15.9,11.9,0.2,-25.4,-4.0,14.5,1.2,4.0,-22.6,5.0,-7.3,8.1,10.9
28,-57.4,-16.3,-19.2,-30.5,10.6,-16.5,3.9,10.8,10.5,-0.2,-11.9,-0.5,16.2,-2.9,-14.4,0.4,13.3,4.0,4.8,4.1,2.1,-3.5,1.5,4.3
32,-47.3,4.8,-9.5,-21.1,-0.0,-2.7,2.9,7.4,13.7,3.6,0.4,4.5,5.8,0.2,8.1,16.4,11.6,7.2,16.0,21.1,17.8,0.8,0.7,5.7
36,-27.4,2.6,3.7,-16.3,8.6,-8.3,3.9,8.4,16.8,-3.1,17.0,9.2,7.8,3.3,8.6,22.7,13.2,3.0,22.8,17.1,23.4,0.0,5.7,5.2
40,-26.1,-2.3,-2.6,-15.2,18.4,-4.7,4.0,5.2,15.5,13.5,13.2,8.4,13.6,8.3,26.7,21.5,9.0,8.6,17.8,17.4,21.8,5.8,8.1,-1.7
44,-19.2,0.5,0.8,-22.2,15.2,-4.3,21.4,6.3,11.1,8.0,15.9,11.2,12.0,10.0,30.3,6.9,14.3,7.3,23.1,26.0,22.9,17.0,7.4,2.0
48,-12.3,1.5,1.0,-26.1,7.8,-0.7,16.4,21.7,13.1,17.7,12.4,9.7,15.7,7.7,22.5,4.8,17.9,22.8,29.9,14.7,30.7,20.8,16.3,10.1


In [4]:
T = np.load('bidask.npz')['T'].view(np.recarray)
T

rec.array([('2023-01-14T12:22:44.180000', 20457.5   , 20457.5996, 0), ('2023-01-14T12:22:44.281000', 20456.4004, 20456.5   , 0),
           ('2023-01-14T12:22:44.378000', 20457.5   , 20457.5996, 0), ('2023-01-14T12:22:44.480000', 20456.8008, 20457.5   , 0),
           ('2023-01-14T12:22:44.576000', 20456.3008, 20456.4004, 0), ..., ('2023-01-30T20:59:02.672000', 23095.8008, 23095.9004, 0),
           ('2023-01-30T20:59:02.899000', 23096.5   , 23096.5996, 0), ('2023-01-30T20:59:03.386000', 23099.4004, 23099.5   , 0),
           ('2023-01-30T20:59:06.868000', 23099.8008, 23099.9004, 0), ('2023-01-30T20:59:06.998000', 23099.9004, 23100.    , 0)],
          dtype=[('DateTime', '<M8[us]'), ('Bid', '<f8'), ('Ask', '<f8'), ('Act', 'i1')])

In [5]:
# Main chart
fig = chartFigure(rows=2, height=450,
                  Bid=dict(y=T['Bid'], color='green', opacity=0.1, shape='hv'),
                  Ask=dict(y=T['Ask'], color='red', opacity=0.1, shape='hv'),
                  Ideal=dict(color='gray', secondary_y=True, shape='hv'),
                  Profit=dict(color='blue', secondary_y=True, shape='hv'),
                  Pred=dict(row=2),
                  Target=dict(row=2, opacity=0.4))

In [6]:
# Profit barchart
fig2 = chartFigure(height=150)
fig2.update_layout(annotations=[Annotation(text='Profit', showarrow=True, arrowhead=3, ax=0, ay=-20)])
fig2.add_trace(Bar(x=np.arange(b, 100), y=R.at[18, 'PnL'][b:], marker_line_color='black', marker_color='green', opacity=0.7));

In [7]:
# Event handlers for update visual controls
def update_annot(th, profit):
    """Update annotation arrow that shows profit at current threshold value"""
    with fig2.batch_update():
        fig2.layout.annotations[0]['x'] = th
        fig2.layout.annotations[0]['y'] = profit


def grid_changed(event, grid):
    """Redraw chart with selected horizon value"""

    changed = grid.get_changed_df()
    k = event['new'][0]
    horizon = changed.iloc[k].name
    
    fig2.data[0].y = R.at[horizon, 'PnL'][b:]
    
    global Pred, Thresholds
    Pred = R.at[horizon, 'Pred']
    Thresholds = np.linspace(0, np.percentile(np.abs(Pred), 99.98), 100)

    th = slider_threshold.value
    p = getProfit(npBacktestMarket(T, Pred, Thresholds[th]))
    updateFigure(fig, 
                 Pred=Pred, Target=R.at[horizon, 'Target'], 
                 Ideal=dict(y=p.MidPnL.cumsum(), x=p.Index),
                 Profit=dict(y=p.Profit.cumsum(), x=p.Index)
                )
    update_annot(th, p.Profit.sum()/T.Bid[0]*100)


def slider_changed(change):
    """Updates profit chart for specified threshold"""
    th = change['new']
    p = getProfit(npBacktestMarket(T, Pred, Thresholds[th]))
    updateFigure(fig, 
                 Ideal=dict(y=p.MidPnL.cumsum(), x=p.Index),
                 Profit=dict(y=p.Profit.cumsum(), x=p.Index)
                )
    update_annot(th, p.Profit.sum()/T.Bid[0]*100)

In [8]:
# Threshold slider
slider_threshold = IntSlider(value=60, min=b, max=99, description='Threshold')
slider_threshold.observe(slider_changed, names='value')

In [9]:
# Result grid
grid = show_grid(r, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False, 'rowHeight': 26, 'maxVisibleRows': 5})
grid.on('selection_changed', grid_changed)
grid.change_selection([18])

In [10]:
# Show all
VBox([slider_threshold, fig, fig2, grid])

    'data…